In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

cols = ['sentiment','id','date','query_string','user','text']

df = pd.read_csv("train.csv",header=None, names=cols, encoding = "ISO-8859-1")
df.drop(['id','date','query_string','user'],axis=1,inplace=True)
df['sentiment'] = df['sentiment'].map({0: 0, 4: 1})


In [5]:
df['pre_clean_len'] = [len(t) for t in df.text]
print(df[df.pre_clean_len > 140].head(10))

      sentiment                                               text  \
213           0  Awwh babs... you look so sad underneith that s...   
226           0  Tuesdayï¿½ll start with reflection ï¿½n then a...   
279           0  Whinging. My client&amp;boss don't understand ...   
343           0  @TheLeagueSF Not Fun &amp; Furious? The new ma...   
400           0  #3 woke up and was having an accident - &quot;...   
464           0  My bathtub drain is fired: it haz 1 job 2 do, ...   
492           0  pears &amp; Brie, bottle of Cabernet, and &quo...   
747           0  Have an invite for &quot;Healthy Dining&quot; ...   
957           0  Damnit I was really digging this season of Rea...   
1064          0  Why do I keep looking...I know that what I rea...   

      pre_clean_len  
213             142  
226             141  
279             145  
343             145  
400             144  
464             146  
492             150  
747             141  
957             141  
1064     

In [12]:
from nltk.tokenize import WordPunctTokenizer
from bs4 import BeautifulSoup
import re
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
pat3 = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom = souped
    stripped = re.sub(combined_pat, '', bom)
    stripped = re.sub(pat3, '', stripped)
    lower_case = stripped.lower()
    neg_handle = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handle)
    words = [x for x in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()


In [13]:
clean_tweets = []
for i in range(0,1600000):
    if( (i+1)%100000 == 0):
        print("tweets %d prrocessed" %(i+1))
    clean_tweets.append(tweet_cleaner(df['text'][i]))

tweets 100000 prrocessed
tweets 200000 prrocessed
tweets 300000 prrocessed
tweets 400000 prrocessed
tweets 500000 prrocessed
tweets 600000 prrocessed
tweets 700000 prrocessed
tweets 800000 prrocessed
tweets 900000 prrocessed
tweets 1000000 prrocessed
tweets 1100000 prrocessed
tweets 1200000 prrocessed
tweets 1300000 prrocessed
tweets 1400000 prrocessed
tweets 1500000 prrocessed
tweets 1600000 prrocessed


In [15]:
df_clean = pd.DataFrame(clean_tweets, columns = ['text'])
df_clean['target'] = df.sentiment
df_clean.head(20)

,text,target
0,awww that bummer you shoulda got david carr of...,0
1,is upset that he can not update his facebook b...,0
2,dived many times for the ball managed to save ...,0
3,my whole body feels itchy and like its on fire,0
4,no it not behaving at all mad why am here beca...,0
5,not the whole crew,0
6,need hug,0
7,hey long time no see yes rains bit only bit lo...,0
8,nope they did not have it,0
9,que me muera,0


In [25]:
df_clean.to_csv('clean_tweet.csv',encoding='utf-8')


In [28]:
np.sum(df_clean.isnull().any(axis=1))


0

In [29]:
df_new = pd.read_csv('clean_tweet.csv',index_col = 0)
df_new.head()

/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,text,target
0,awww that bummer you shoulda got david carr of...,0
1,is upset that he can not update his facebook b...,0
2,dived many times for the ball managed to save ...,0
3,my whole body feels itchy and like its on fire,0
4,no it not behaving at all mad why am here beca...,0


In [30]:
np.sum(df_new.isnull().any(axis=1))


3478

In [32]:
df_new.dropna(inplace=True)
df_new.reset_index(drop=True,inplace=True)
np.sum(df_new.isnull().any(axis=1))


0